In [7]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## Import data

In [14]:
%cd /content/drive/MyDrive/nlp-getting-started/
!ls

/content/drive/MyDrive/nlp-getting-started
sample_submission.csv  test.csv  train.csv


## Explore data

In [27]:
train_df = pd.read_csv("train.csv")
print(train_df.iloc[0])
print(train_df[train_df["target"] == 0]["text"].values[0])
print(train_df[train_df["target"] == 1]["text"].values[0])

id                                                          1
keyword                                                   NaN
location                                                  NaN
text        Our Deeds are the Reason of this #earthquake M...
target                                                      1
Name: 0, dtype: object
What's up man?
Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
